<center><h1>Cultural Analytics: Homework #3</h1></center>

<center><b>Due</b> 11:59PM 10/13/2019</center>


---

In [ ]:
# Let's begin by loading up some libraries
import os
import csv
import numpy as np
import sklearn
import pickle
from nltk.tokenize import word_tokenize
from tsvdro import tsvdro
from sklearn.feature_extraction import DictVectorizer

### Part One: Similarity with Close Reading

You'll need to locate a sonnet or some other object of similar length (fourteen lines) and word count. Select and "cut" this into a buffer to paste below.

In [ ]:
# Here are going to load some previously saved data.
# These are sonnets from eighteenth century sources. Many of these
# are much older than the eighteenth century, however.
#

fp = open('data/mmlec/labeled_objects_mmlec_sonnets.pkl','rb')
mmlec_sonnets = pickle.load(fp)

# how many sample sonnets do we have?
len(mmlec_sonnets)

In [ ]:
# now cut and past a sample poem or section of text into the space below
sample_sonnet = """

"""

# tokenize (for later) and also add to our data
sample_tokens = word_tokenize(sample_sonnet)
mmlec_sonnets.append(sample_sonnet)

In [ ]:
# we're dealing with small objects, so let's try the binary vector method

# Import CountVectorizer and configure options
# to learn more about the various options, see the
# documentation for this function:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

from sklearn.feature_extraction import text
vectorizer = text.CountVectorizer(input='content',
                                  lowercase='true',
                                  binary='true',
                                  strip_accents='unicode')

# This does the actual vectorization
counts = vectorizer.fit_transform(mmlec_sonnets)

# get vocabulary for later
vocabulary = vectorizer.get_feature_names()

# Return total number of documents and the number of items in the vocabulary
dc, vc = counts.shape
print("document count:",dc,"vocabulary count:",vc)

In [ ]:
# calculate distance between each sonnet
from sklearn.metrics.pairwise import cosine_similarity
cosine_dist_matrix = 1 - cosine_similarity(counts)

In [ ]:
# find similar texts

difference_list=dict()
sample_text = len(mmlec_sonnets) - 1
for i, text in enumerate(mmlec_sonnets): 
    difference_list[i] = np.round(cosine_dist_matrix[sample_text,i],3)
    i += 1
    
# display sorted distances in vector space from our selected text
x = 0
for i in sorted(difference_list, key=difference_list.__getitem__):
    if x < 100:
        print("\n------------------------------------------------------\n")
        matched_tokens = word_tokenize(mmlec_sonnets[i])
        for word in matched_tokens:
            if word in sample_tokens and word in vocabulary:
                print("*{0}*".format(word),end = ' ')
            else:
                print(word, end = ' ')
        x += 1

<b> Question #1:</b> What observations can you make about the sonnets returned as most similar (they are returned in order of similarity)? What is being returned? Explain what you take to be the signficance of the returned data.

### Part Two: Plotting Genre

The following cells will read in Ted Underwood's metadata for Chapter 1 and then plot/visualize these genres. You will need to select a number of texts to pick from each genre. Replace 'XXX' with a number for the n_documents variable. Execute/Run the remainder of the cells.

In [ ]:
# This lines tell the Python interpreter to load some additional functions.
# We need functions to parse Comma Separated Value (CSV) data.
import csv

# we'll set a variable to hold the number of records read from this CSV file.
ln = 0

# as above, we'll create a metadata variable (overwritting the previous variable)
metadata=list()

# Now we open the CSV file and read each line, appending to the metadata list:
with open('data/Underwood_ch1/allgenremeta.csv', encoding = 'utf-8') as f:
    reader = csv.reader(f, delimiter = ',')
    for row in reader:
        metadata.append(row)
        
        # increment our counter
        ln += 1

# tell us how many entries we've read
print("read %s lines" % ln)

# Let's remove the header (our "metadata") for easier processing:
metadata = metadata[1:]

In [ ]:
# Select:
#
# fifty works of fiction
# fifty works of poetry
# fifty works of biography

# How many texts per category?
n_documents = XXX


selected_documents=list()
selected_metadata=list()

counters=dict()
counters['bio'] = 0
counters['fic'] = 0
counters['poe'] = 0

def load_and_add(short_name):
    fn = 'data/Underwood_ch1/' + short_name + ".dro"
    dro = tsvdro.load(fn)

    selected_documents.append(dro['data'])
    selected_metadata.append(dro['header'])
    
for document in metadata:
    if document[5] == "bio" and counters['bio'] < n_documents:
            load_and_add(document[0])
            counters['bio'] += 1
    if document[5] == "fic" and counters['fic'] < n_documents:
            load_and_add(document[0])
            counters['fic'] += 1
    if document[5] == "poe" and counters['poe'] < n_documents:
            load_and_add(document[0])
            counters['poe'] += 1

In [ ]:
vectorizer = DictVectorizer()
counts = vectorizer.fit_transform(selected_documents)

In [ ]:
# Return total number of documents and the number of items in the vocabulary
dc, vc = counts.shape
print("document count:",dc,"vocabulary count:",vc)

In [ ]:
counts = counts.toarray()
from sklearn.metrics.pairwise import cosine_similarity
cosine_dist_matrix = 1 - cosine_similarity(counts)

In [ ]:
from sklearn.manifold import MDS
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
pos = mds.fit_transform(cosine_dist_matrix)

In [ ]:
# visualize.
# 
# NOTE: you might need to click on the image to expand it enough to read it.
#
import matplotlib.pyplot as plt
xs, ys = pos[:, 0], pos[:, 1]
fig = plt.figure(figsize=(35, 20), dpi=75)
genres = [x['bibliographic_data']['genre'] for x in selected_metadata]
for x, y, genre in zip(xs, ys, genres):
    plt.scatter(x, y, c='black',s=10)
    plt.text(x, y, genre)
plt.axis('off')
plt.show()

<b>Question #2:</b>: Examine the above genre "map." Can you separate poetry from fiction from biography? What might this mean? 